In [1]:
import asyncio
import json
import pandas as pd
from time import sleep
import pprint
import hmac
import hashlib
from datetime import datetime
import nest_asyncio
import numpy as np
import websockets


from keys import URL, CLIENT_ID, CLIENT_SECRET

nest_asyncio.apply()

In [2]:
url = URL
clientId = CLIENT_ID
clientSecret = CLIENT_SECRET

In [45]:
auth_msg = {
    "jsonrpc":"2.0",
    "id":42,
    "method":"public/auth",
    "params":{
        "grant_type":"client_credentials",
        "client_id" : clientId,
        "client_secret" : clientSecret
        
    }
    
}

order_book_msg = {
    "jsonrpc": "2.0",
    "id": 42,
    "method": "public/get_order_book",
    "params": {
        "instrument_name": "BTC-2FEB24",
        "depth": 30
    }
}

cancel_msg = {
    "jsonrpc" : "2.0",
    "id" : 42,
    "method" : "private/cancel_all_by_instrument",
    "params" : {
    "instrument_name" : "BTC-2FEB24" 
    }
}

In [29]:
async def pub_api(msg):
        async with websockets.connect(url) as websocket:
            await websocket.send(msg)
            while websocket.open:
                response = await websocket.recv()
                return json.loads(response)
        
async def priv_api(msg):
        async with websockets.connect(url) as websocket:
            await websocket.send(json.dumps(auth_msg))
            while websocket.open:
                response = await websocket.recv()
                await websocket.send(msg)
                response = await websocket.recv()
                break
            return json.loads(response)

In [42]:
book = asyncio.get_event_loop().run_until_complete(pub_api(json.dumps(order_book_msg)))

In [43]:
book['result']['bids'][0][0]

42750.0

In [36]:
def order_msg(price,side,weight):
    return {
      "jsonrpc" : "2.0",
      "id" : 42,
      "method" : "private/"+side,
      "params" : {
        "instrument_name" : "BTC-2FEB24",
        "amount" : round(weight),
        "type" : "limit",
        "label" : "order_"+side,
        "price" : price,
        "time_in_force": "good_til_cancelled",
        "post_only": True
      }
    }

In [48]:
cancel = asyncio.get_event_loop().run_until_complete(priv_api(json.dumps(cancel_msg)))

In [51]:
# set a limit order
book = asyncio.get_event_loop().run_until_complete(pub_api(json.dumps(order_book_msg)))

# limit_order = asyncio.get_event_loop().run_until_complete(priv_api(json.dumps(order_msg(book['result']['asks'][0][0], 'buy', 10))))

limit_order = asyncio.get_event_loop().run_until_complete(priv_api(json.dumps(order_msg(book['result']['asks'][0][0], 'sell', 10))))

# limit_order = asyncio.get_event_loop().run_until_complete(priv_api(json.dumps(order_msg(1, 'sell', 10))))